# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [57]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [58]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("./Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(40)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [59]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [60]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [61]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [62]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [63]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

In [64]:
# Create an empy list to store the inertia values
inertia = []

In [65]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)
    
inertia

C:\Users\ToddGarner\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "


[287.0,
 195.82021818036043,
 123.19048183836959,
 79.02243535120975,
 65.302379141625,
 52.888518219775314,
 43.91469044757747,
 37.517032490103574,
 32.485240832183536,
 28.22289929060893]

In [66]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k,
    "inertia": inertia
}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data = pd.DataFrame(elbow_data)


In [67]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow_data.hvplot.line(
    x="k", 
    y= "inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [68]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)


In [69]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)


KMeans(n_clusters=4)

In [70]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model.predict(df_market_data_scaled)

# View the resulting array of cluster values.
print(crypto_clusters)

[2 2 1 1 2 2 2 2 2 1 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 1 1 1 2 1 1 1 0 2 1 1 3
 1 1 1 1]


In [71]:
# Create a copy of the DataFrame
df_market_data_predictions = df_market_data_scaled.copy()

In [72]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_predictions["CryptoCluster"] = crypto_clusters

# Display sample data
df_market_data_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,CryptoCluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [73]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_24h"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_data_predictions.hvplot.scatter(x='price_change_percentage_24h', y='price_change_percentage_24h', by='coin_id', 
                  legend='top', height=400, width=400,
                  hover_cols=["all"])

:NdOverlay   [coin_id]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_24h)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [74]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [75]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca_data = pca.fit_transform(df_market_data_predictions)

# View the first five rows of the DataFrame. 
market_pca_data[:5]

array([[-1.19018713,  0.04818215,  0.46848144],
       [-0.82944857, -0.03220451,  0.99049453],
       [ 0.01014993, -0.50588309, -0.67646595],
       [ 0.02452739, -0.56428507, -0.51140341],
       [-2.49908534,  0.26382132,  1.72364204]])

In [76]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.36384149, 0.35308123, 0.17298865])

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.8881101751527735


In [77]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data
df_market_pca_data = pd.DataFrame(
    market_pca_data,
    columns=["PC1", "PC2", "PC3"])
# Copy the crypto names from the original data
df_market_data_names = df_market_pca_data.copy()

# Set the coinid column as index
df_market_data_names["coin_id"] = df_market_data.index
# Display sample data
print(df_market_data_names)

         PC1       PC2       PC3               coin_id
0  -1.190187  0.048182  0.468481               bitcoin
1  -0.829449 -0.032205  0.990495              ethereum
2   0.010150 -0.505883 -0.676466                tether
3   0.024527 -0.564285 -0.511403                ripple
4  -2.499085  0.263821  1.723642          bitcoin-cash
5  -1.584926  0.420229  0.753308           binancecoin
6  -0.949500  0.043913  2.848072             chainlink
7  -1.003391  0.219762  1.485883               cardano
8  -0.899487 -0.212729  0.641963              litecoin
9  -0.163630 -0.785357 -0.258678       bitcoin-cash-sv
10  1.096270 -1.024367 -1.367207      crypto-com-chain
11  0.013764 -0.514669 -0.697181              usd-coin
12  0.101181 -0.885658 -0.538956                   eos
13 -2.491684  1.739987  1.305358                monero
14 -0.039532 -0.428417 -0.064155                  tron
15 -0.272110 -0.827865  1.201709                 tezos
16  1.171456 -0.700269 -1.016105                   okb
17  0.3985

---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [78]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

In [79]:
# Create an empy list to store the inertia values
inertia = []

In [80]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_pca_data)
    inertia.append(model.inertia_)
    
inertia

C:\Users\ToddGarner\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "


[267.16875671760886,
 176.21422095646767,
 99.60800289653415,
 47.29227876924115,
 35.9779434994357,
 26.788723532904687,
 20.648813155994986,
 16.97647395212377,
 13.585860135279104,
 10.354459545869439]

In [81]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data2 = {
    "k": k,
    "inertia": inertia
}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data2 = pd.DataFrame(elbow_data2)


In [82]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow_data2.hvplot.line(
    x="k", 
    y= "inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It doesn't differ

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [83]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)


In [84]:
# Fit the K-Means model using the PCA data
model.fit(df_market_pca_data)

KMeans(n_clusters=4)

In [85]:
# Predict the clusters to group the cryptocurrencies using the PCA data
market_clusters = model.predict(df_market_pca_data)

# View the resulting array of cluster values.
print(market_clusters)

[3 3 0 0 3 3 3 3 3 0 0 0 0 3 0 3 0 0 3 0 0 3 0 0 0 0 0 0 3 0 0 0 1 3 0 0 2
 0 0 0 0]


In [86]:
# Create a copy of the DataFrame with the PCA data

# Add a new column to the DataFrame with the predicted clusters
df_market_pca_data["MarketCluster"] = market_clusters

df_market_pca_data["Coin_ID"] = df_market_data.index

# Display sample ddf_market_data_pca1 = df_market_data_pca.copy()

df_market_pca_data

,PC1,PC2,PC3,MarketCluster,Coin_ID
0,-1.190187,0.048182,0.468481,3,bitcoin
1,-0.829449,-0.032205,0.990495,3,ethereum
2,0.010150,-0.505883,-0.676466,0,tether
3,0.024527,-0.564285,-0.511403,0,ripple
4,-2.499085,0.263821,1.723642,3,bitcoin-cash
5,-1.584926,0.420229,0.753308,3,binancecoin
6,-0.949500,0.043913,2.848072,3,chainlink
7,-1.003391,0.219762,1.485883,3,cardano
8,-0.899487,-0.212729,0.641963,3,litecoin
9,-0.163630,-0.785357,-0.258678,0,bitcoin-cash-sv


In [87]:
df_market_pca_data.describe()

,PC1,PC2,PC3,MarketCluster
count,4.100000e+01,4.100000e+01,4.100000e+01,41.00000
mean,-1.364085e-16,2.707861e-17,-7.311225e-17,1.02439
std,1.652516e+00,1.627896e+00,1.139458e+00,1.40513
min,-3.030345e+00,-1.724930e+00,-2.252749e+00,0.00000
25%,-8.432498e-01,-7.718479e-01,-6.950084e-01,0.00000
50%,-1.257819e-02,-5.058831e-01,-1.403115e-01,0.00000
75%,1.140113e-01,4.818215e-02,6.419629e-01,3.00000
max,7.821883e+00,7.820059e+00,2.848072e+00,3.00000


In [88]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_pca_data.hvplot.scatter(x='PC1', y='PC2', by='Coin_ID', 
                  legend='top', height=400, width=400,
                  hover_cols=["Coin_ID"])

:NdOverlay   [Coin_ID]
   :Scatter   [PC1]   (PC2,Coin_ID)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [89]:
# Composite plot to contrast the Elbow curves
df_combo = (df_elbow_data + df_elbow_data2)
"""
What's interesting is that the lesson 3 Module 6 shows multiplying the two dataframes to overlay them but adding the dataframes together gives me the same result.  

"""

df_combo.hvplot.line(
    x="k", 
    y= "inertia", 
    title="Combination Elbow Curve", 
    legend='top', height=500, width=1200,
    xticks=k
)

:Curve   [k]   (inertia)

In [90]:
# Compoosite plot to contrast the clusters
df_combo = df_market_pca_data.hvplot.scatter(x='PC1', y='PC2', by='Coin_ID', legend='top', height=400, width=400, hover_cols=["Coin_ID"]) + df_market_data_predictions.hvplot.scatter(x='price_change_percentage_24h', y='price_change_percentage_24h', by='coin_id', legend='top', height=400, width=400, hover_cols=["all"])


df_combo

:Layout
   .NdOverlay.I  :NdOverlay   [Coin_ID]
      :Scatter   [PC1]   (PC2,Coin_ID)
   .NdOverlay.II :NdOverlay   [coin_id]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_24h)

In [91]:
df_combo = df_market_pca_data.hvplot.scatter(x='PC1', y='PC2', by='Coin_ID', legend='top', height=400, width=400, hover_cols=["Coin_ID"]) * df_market_data_predictions.hvplot.scatter(x='price_change_percentage_24h', y='price_change_percentage_24h', by='coin_id', legend='top', height=400, width=400, hover_cols=["all"])


df_combo

:Overlay
   .NdOverlay.I  :NdOverlay   [Coin_ID]
      :Scatter   [PC1]   (PC2,Coin_ID)
   .NdOverlay.II :NdOverlay   [coin_id]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_24h)

In [92]:
df_combo = df_market_pca_data.hvplot.scatter(x='PC1', y='PC2', by='Coin_ID', legend='top', height=400, width=400, hover_cols=["Coin_ID"]) + df_market_data_predictions.hvplot.scatter(x='price_change_percentage_24h', y='price_change_percentage_24h', by='coin_id', legend='top', height=400, width=400, hover_cols=["all"])


df_combo

:Layout
   .NdOverlay.I  :NdOverlay   [Coin_ID]
      :Scatter   [PC1]   (PC2,Coin_ID)
   .NdOverlay.II :NdOverlay   [coin_id]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_24h)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** When looking at the columns of data in the prior dataframe from which we started, there is no way to graph a dataframe with more than 3 columns.  By reducing the number of columns but not the data contained within, we can use visualization tools to pick out these coins below from one of the quadrants in the scatter plot to make our selections for the crypto currencies to recommend to the board.  I understand now how these methods can bring in visualization as a critical part of the overall analysis.  Thank you again for giving me the opportunity to repair this work and to learn its value.  

In [94]:
"""
1. Monero*
2. bitcoin-cash*
3. binancecoin*
4. wrapped bitcoin*
5. cosmos*
6. cardano*
7. zcash*
8. omisego*
9. celsius-degree-token*
10. theta-token*
11. ethland*
12. nem*
"""


'\n1. Monero\n2. bitcoin-cash\n3. binancecoin\n4. wrapped bitcoin\n5. cosmos\n6. cardano\n7. zcash\n8. ethereum\n9. litecoin\n10. chainlink\n11. omisego\n12. celsius-degree-token\n13. ftx-token\n14. maker\n15. USD coin\n16. ripple\n'